In [0]:
%pip install web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 36.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Troubleshooting steps

In [0]:
from web3 import Web3

provider_uri = "https://mainnet.infura.io/v3/6504e6a7883c4b49ac1cf17099e2ea3a"
w3 = Web3(Web3.HTTPProvider(provider_uri))

try:
    connected = w3.is_connected()
    print(f"Connected: {connected}")
    if connected:
        latest = w3.eth.block_number
        print(f"Latest block: {latest}")
        print("Connection is working!")
    else:
        print("Connection failed - not connected")
except Exception as e:
    print(f"Error: {e}")

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("EthereumStream").getOrCreate()

# Test if registration works
try:
    spark.dataSource.register(EthereumDataSource)
    print("✓ DataSource registered successfully")
except Exception as e:
    print(f"✗ Registration failed: {e}")

# List registered data sources to verify
print("\nRegistered data sources:")
# This may not work on all Spark versions, but try it
try:
    print(spark.catalog.listTables())
except:
    pass

# Test instantiation directly
try:
    test_options = {
        "provider_uri": "https://mainnet.infura.io/v3/6504e6a7883c4b49ac1cf17099e2ea3a",
        "start_block": "19000000"
    }
    test_source = EthereumDataSource(test_options)
    print(f"✓ DataSource instantiated: {test_source.name()}")
    print(f"✓ Schema: {test_source.schema()}")
except Exception as e:
    print(f"✗ Instantiation failed: {e}")

# Now try the stream
checkpoint_path = "/Volumes/blockchain/ethereum/checkpoints/"
output_path = "/Volumes/blockchain/ethereum/output/"

print(f"\nCheckpoint: {checkpoint_path}")
print(f"Output: {output_path}")

try:
    df = spark.readStream \
        .format("ethereum") \
        .option("provider_uri", "https://mainnet.infura.io/v3/6504e6a7883c4b49ac1cf17099e2ea3a") \
        .option("start_block", "19000000") \
        .load()
    
    print("✓ Stream created")
    print(f"Schema: {df.schema}")
    
except Exception as e:
    print(f"✗ Stream creation failed: {e}")
    import traceback
    traceback.print_exc()

In [0]:
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder.appName("EthereumStream").getOrCreate()

checkpoint_path = "/Volumes/blockchain/ethereum/checkpoints/"
output_path = "/Volumes/blockchain/ethereum/output/"

print(f"Checkpoint: {checkpoint_path}")
print(f"Output: {output_path}")

spark.dataSource.register(EthereumDataSource)

df = spark.readStream \
    .format("ethereum") \
    .option("provider_uri", "https://mainnet.infura.io/v3/6504e6a7883c4b49ac1cf17099e2ea3a") \
    .option("start_block", "19000000") \
    .option("max_calls_per_second", "0.5") \
    .option("batch_size", "5") \
    .load()

print("✓ Stream DataFrame created")

# Now try to start the write
try:
    query = df.writeStream \
        .format("delta") \
        .outputMode("append") \
        .option("checkpointLocation", checkpoint_path) \
        .trigger(processingTime="30 seconds") \
        .start(output_path)
    
    print("✓ Write stream started!")
    print(f"Query ID: {query.id}")
    print(f"Query name: {query.name}")
    
    # Wait a bit and check status
    time.sleep(10)
    
    print(f"\nQuery is active: {query.isActive}")
    print(f"Query status: {query.status}")
    
    if query.exception():
        print(f"\n✗ Query has exception: {query.exception()}")
    else:
        print("\n✓ No exceptions yet")
    
    # Check progress
    progress = query.recentProgress
    if progress:
        print(f"\nRecent progress: {len(progress)} batches")
        for i, p in enumerate(progress[-3:]):  # Last 3 batches
            print(f"  Batch {i}: {p}")
    
    # Don't block - let it run in background
    # query.awaitTermination()
    
except Exception as e:
    print(f"\n✗ Failed to start write stream: {e}")
    import traceback
    traceback.print_exc()

In [0]:
# Check what latestOffset is returning

print("\nManually testing the streaming reader:")
test_reader = EthereumStreamReader(
    schema=StructType([
        StructField("block_number", LongType()),
        StructField("block_hash", StringType()),
        StructField("timestamp", LongType()),
        StructField("tx_count", LongType())
    ]),
    options={
        "provider_uri": "https://mainnet.infura.io/v3/6504e6a7883c4b49ac1cf17099e2ea3a",
        "start_block": "19000000",
        "batch_size": "5"
    }
)

print("\nTesting initialOffset:")
initial = test_reader.initialOffset()
print(f"Initial offset: {initial}")

print("\nTesting latestOffset:")
try:
    latest = test_reader.latestOffset()
    print(f"Latest offset: {latest}")
except Exception as e:
    print(f"latestOffset FAILED: {e}")

print("\nTesting partitions:")
try:
    parts = test_reader.partitions(initial, latest)
    print(f"Number of partitions created: {len(parts)}")
    if parts:
        print(f"First partition: blocks {parts[0].start} to {parts[0].end}")
except Exception as e:
    print(f"partitions FAILED: {e}")

print("\nTesting read on first partition:")
try:
    if parts:
        rows = list(test_reader.read(parts[0]))
        print(f"Read {len(rows)} rows")
        if rows:
            print(f"First row: {rows[0]}")
except Exception as e:
    print(f"read FAILED: {e}")

**Main code starts**....

In [0]:

import os, json, time, logging
from pyspark.sql.datasource import DataSource, DataSourceStreamReader, InputPartition
from pyspark.sql.types import StructType, StructField, LongType, StringType
from pyspark.sql import Row
from web3 import Web3

            

# -----------------------------
# Logging setup
# -----------------------------
logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s [%(levelname)s] [%(name)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger("EthereumStream")

# -----------------------------
# Partition class
# -----------------------------
class BlockRangePartition(InputPartition):
    def __init__(self, start, end):
        self.start = start
        self.end = end
        logger.debug(f"Created partition: blocks {start} to {end}")

# -----------------------------
# StreamReader
# -----------------------------
class EthereumStreamReader(DataSourceStreamReader):
    def __init__(self, schema, options):
        logger.info("=" * 60)
        logger.info("Initializing EthereumStreamReader")
        logger.info("=" * 60)
        
        self.schema = schema
        self.options = options
        self.provider_uri = options.get("provider_uri")
        self.start_block = int(options.get("start_block", 1))
        self.output_dir = options.get("output_dir", "dbfs:/Volumes/blockchain/ethereum/blocks/raw")
        self.current_block = self.start_block
        self.max_calls_per_second = float(options.get("max_calls_per_second", 0.8))

        logger.info(f"Configuration:")
        logger.info(f"  - Provider URI: {self.provider_uri}")
        logger.info(f"  - Start block: {self.start_block}")
        logger.info(f"  - Output directory: {self.output_dir}")
        logger.info(f"  - Max calls/sec: {self.max_calls_per_second}")
        logger.info("=" * 60)

    # -------------------------
    # Offset handling (dict-based)
    # -------------------------
    def initialOffset(self):
        offset = {"offset": self.current_block}
        logger.info(f"initialOffset() called -> Returning: {offset}")
        return offset

    def latestOffset(self):
        logger.info("latestOffset() called -> Connecting to Ethereum provider...")
        try:
            w3 = Web3(Web3.HTTPProvider(self.provider_uri))
            if not w3.is_connected():
                logger.error("Failed to connect to Ethereum provider in latestOffset()")
                raise RuntimeError("Failed to connect to Ethereum provider")
            
            latest = w3.eth.block_number
            offset = {"offset": latest}
            logger.info(f"latestOffset() -> Latest block from chain: {latest}")
            logger.info(f"latestOffset() -> Returning: {offset}")
            return offset
        except Exception as e:
            logger.error(f"Error in latestOffset(): {e}", exc_info=True)
            raise

    # -------------------------
    # Partition planning
    # -------------------------
    def partitions(self, start_offset, end_offset):
        """Legacy method name - calls planPartitions internally"""
        return self.planPartitions(start_offset, end_offset)
    
    def planPartitions(self, start_offset, end_offset):
        logger.info("-" * 60)
        logger.info("planPartitions() called")
        logger.info(f"  - start_offset: {start_offset}")
        logger.info(f"  - end_offset: {end_offset}")
        
        start = start_offset.get("offset", self.start_block) if start_offset else self.start_block
        end = end_offset.get("offset", start + 100) if end_offset else start + 100
        step = int(self.options.get("batch_size", 10))
        
        logger.info(f"  - Computed start: {start}")
        logger.info(f"  - Computed end: {end}")
        logger.info(f"  - Batch size: {step}")
        
        partitions = []
        for i in range(start, end + 1, step):
            partition_end = min(i + step - 1, end)
            partitions.append(BlockRangePartition(i, partition_end))
        
        logger.info(f"  - Created {len(partitions)} partition(s)")
        for idx, p in enumerate(partitions):
            logger.info(f"    Partition {idx}: blocks {p.start} to {p.end} ({p.end - p.start + 1} blocks)")
        logger.info("-" * 60)
        
        return partitions

    # -------------------------
    # Reader
    # -------------------------
    def read(self, partition):
        logger.info("*" * 60)
        logger.info(f"read() called for partition: blocks {partition.start} to {partition.end}")
        logger.info("*" * 60)
        
        try:
            w3 = Web3(Web3.HTTPProvider(self.provider_uri))
            if not w3.is_connected():
                logger.error("Failed to connect to Ethereum provider in read()")
                raise RuntimeError("Failed to connect to Ethereum provider")
            
            logger.info("Successfully connected to Ethereum provider")
        except Exception as e:
            logger.error(f"Connection error in read(): {e}", exc_info=True)
            raise

        last_call_time = 0
        blocks_processed = 0
        blocks_failed = 0
        
        for blk_num in range(partition.start, partition.end + 1):
            try:
                # Rate limiting
                now = time.time()
                sleep_time = (1 / self.max_calls_per_second) - (now - last_call_time)
                if sleep_time > 0:
                    logger.debug(f"Rate limiting: sleeping for {sleep_time:.3f}s")
                    time.sleep(sleep_time)
                last_call_time = time.time()

                logger.info(f"Fetching block {blk_num}...")
                block = w3.eth.get_block(blk_num, full_transactions=False)

                blk_dict = {
                    "block_number": block.number,
                    "block_hash": block.hash.hex(),
                    "timestamp": block.timestamp,
                    "tx_count": len(block.transactions)
                }

                # Save JSON locally (optional)
                file_path = os.path.join(self.output_dir, f"block_{blk_num}.json")
                with open(file_path, "w") as f:
                    json.dump(blk_dict, f, indent=2)
                logger.debug(f"Saved block data to {file_path}")

                logger.info(f"✓ Block {blk_num}: hash={block.hash.hex()[:10]}..., "
                           f"timestamp={block.timestamp}, tx_count={len(block.transactions)}")

                yield Row(
                    block_number=block.number,
                    block_hash=block.hash.hex(),
                    timestamp=block.timestamp,
                    tx_count=len(block.transactions)
                )
                
                self.current_block = blk_num + 1
                blocks_processed += 1

            except Exception as e:
                blocks_failed += 1
                logger.error(f"✗ Error fetching block {blk_num}: {e}", exc_info=True)
                continue
        
        logger.info("*" * 60)
        logger.info(f"read() completed for partition {partition.start}-{partition.end}")
        logger.info(f"  - Blocks processed: {blocks_processed}")
        logger.info(f"  - Blocks failed: {blocks_failed}")
        logger.info("*" * 60)

    # -------------------------
    # Commit method for checkpointing
    # -------------------------
    def commit(self, end_offset):
        logger.info("=" * 60)
        logger.info(f"commit() called with end_offset: {end_offset}")
        logger.info(f"Successfully committed up to block: {end_offset.get('offset')}")
        logger.info("=" * 60)

    # -------------------------
    # Pickle support
    # -------------------------
    def __getstate__(self):
        logger.debug("__getstate__() called for serialization")
        state = self.__dict__.copy()
        return state

    def __setstate__(self, state):
        logger.debug("__setstate__() called for deserialization")
        self.__dict__.update(state)

# -----------------------------
# DataSource wrapper
# -----------------------------
class EthereumDataSource(DataSource):
    def __init__(self, options):
        logger.info("=" * 60)
        logger.info("EthereumDataSource.__init__() called")
        logger.info(f"Options: {options}")
        logger.info("=" * 60)
        self.options = options

    @classmethod
    def name(cls):
        logger.debug("name() called -> returning 'ethereum'")
        return "ethereum"

    def schema(self):
        schema = StructType([
            StructField("block_number", LongType()),
            StructField("block_hash", StringType()),
            StructField("timestamp", LongType()),
            StructField("tx_count", LongType())
        ])
        logger.info("schema() called -> returning schema with 4 fields")
        logger.debug(f"Schema: {schema}")
        return schema

    def streamReader(self, schema):
        logger.info("streamReader() called -> creating EthereumStreamReader")
        return EthereumStreamReader(schema, self.options)



In [0]:

#import uuid
import time
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType
from web3 import Web3

# --- Spark session ---
spark = SparkSession.builder.appName("EthereumStream").getOrCreate()

# --- Dynamic checkpoint/output paths ---
base_checkpoint = "/Volumes/blockchain/ethereum/checkpoints"
base_output = "/Volumes/blockchain/ethereum/output"


eth_schema = StructType([
    StructField("block_number", LongType(), True),
    StructField("block_hash", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("tx_count", LongType(), True)
])


checkpoint_path = f"{base_checkpoint}/"
output_path = f"{base_output}/"

print("✅ Using checkpoint:", checkpoint_path)
print("✅ Using output:", output_path)

spark.dataSource.register(EthereumDataSource)

from pyspark.sql.types import StructType, StructField, LongType, StringType, ArrayType, MapType


# Connect to an Ethereum node (Infura, Alchemy, etc.)
w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/6504e6a7883c4b49ac1cf17099e2ea3a"))

# Get latest block number
start_block = w3.eth.block_number
print("Latest Ethereum block number:", start_block)



# --- Read from custom Ethereum source ---
df = spark.readStream \
    .format("ethereum") \
    .schema(eth_schema) \
    .option("provider_uri", "https://mainnet.infura.io/v3/6504e6a7883c4b49ac1cf17099e2ea3a") \
    .option("start_block", start_block) \
    .option("poll_interval", 15) \
    .load()

# --- Write with dynamic paths ---
query = df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_path) \
    .trigger(availableNow=True) \
    .start(output_path)

#availableNow
logger.info("Streaming query started. Awaiting termination...")


✅ Using checkpoint: /Volumes/blockchain/ethereum/checkpoints/
✅ Using output: /Volumes/blockchain/ethereum/output/
Latest Ethereum block number: 23516091


2025-10-06 03:37:55 [INFO] [EthereumStream] Streaming query started. Awaiting termination...
